### Uncertainty analysis
Using the Monte Carlo Simulations method

In [6]:
import numpy as np
import rasterio as rio

In [7]:
# Load the rasters
ksat = rio.open('Rasters/ksat.tif')
wcs = rio.open('Rasters/wcs.tif')
alt_dev = rio.open('Rasters/alt-dev.tif')

In [8]:
# Convert rasters to numpy arrays
ksat_data = ksat.read()
wcs_data = wcs.read()
alt_dev_data = alt_dev.read()


In [9]:
weights = [0.43, 0.14, 0.43]

# Define the type for each criterion
# '1' for benefit, '0' for cost
types = [1, 1, 0]

In [ ]:
result = smca_model(
    ksat_data,
    wcs_data,
    alt_dev_data,
    weights,
    types
)


In [ ]:
monte_carlo_